# STEP 0: Install dependencies

In [ ]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai


# STEP 1: Imports and secure API key input

In [ ]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [ ]:
os.environ['Google_API_key']=getpass.getpass("enter your gemini api key")

# STEP 2: Initialize Gemini 1.5 Flash

In [ ]:

llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2, google_api_key=GOOGLE_API_KEY)



# STEP 3: Node to ask user for symptom

In [ ]:
def get_symptoms(state: dict)-> dict:
  symptom=input("welcome to hospital enter your symptom")
  state["symptom"]= symptom
  return state

# STEP 4: Node to classify the symptom

In [ ]:
def classify_symptom(state: dict)-> dict:
 prompt =(
     "Act like a health agent yourduty is to analyze the symptom that given to you \n"
     "General\n -Emergency \n -mental health \n"
     f"symptom: {state['symptom']} \n"
     "respond only with one word : General or emergency or mental health"

 )
 response =llm.invoke([HumanMessage(content=prompt)])
 category=response.content.strip()
 print(f"LLM classifies the symptom as : {category}")
 state["category"]=category
 return state

# STEP 5: Router logic to route to the correct node


In [ ]:
def symptom_router(state:dict) -> dict:
 cat=state["category"].lower()
 if "general" in cat:
  return "general"
 elif "mental health" in cat:
   return "mental health"
 else:
    return "emergency"

# STEP 6: Category-specific response nodes


In [ ]:
def general_node(state: dict)->dict:
  state["answer"] =f"{state ['symptom']}' :seems general:directingyou to general ward for consulting a doctor"
  return state
def mental_node(state: dict)-> dict:
  state["answer"] =f"{state ['symptom']}' :seems Mental:directingyou to go Mental ward for immediate admit"
  return state
def emergency_node(state: dict)-> dict:
  state["answer"] =f"{state ['symptom']}' :seems Emergency:directingyou to go emergency ward for immidiate admit"
  return state


# STEP 7: Build LangGraph


In [ ]:
from langgraph.graph import StateGraph, END

builder = StateGraph(dict)
builder.set_entry_point("get_symptoms")
builder.add_node("get_symptoms",get_symptoms)
builder.add_node("classify_symptom",classify_symptom)
builder.add_node("general",general_node)
builder.add_node("mental health",mental_node)
builder.add_node("emergency",emergency_node)

builder.add_conditional_edges("classify_symptom",symptom_router,{
  "general":"general",
  "mental health":"mental health",
  "emergency":"emergency"
})
builder.add_edge("get_symptoms","classify_symptom")
builder.add_edge("general",END)
builder.add_edge("mental health",END)
builder.add_edge("emergency",END)

# STEP 8: Compile and invoke the graph


In [ ]:
graph = builder.compile()
final_State=graph.invoke({})
print("final Output \n")
print(final_State["answer"])